In [339]:
import random
from enum import Enum
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,AveragePooling2D
import tensorflow
from IPython.display import display
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [369]:
import itertools
import numpy as np
def testAndTrain():
    train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range =0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip =True)
    test_datagen = ImageDataGenerator(rescale = 1./255)
                                      
    training_set = train_datagen.flow_from_directory('dogs-vs-cats/',
                                                target_size=(64,64),
                                                batch_size= 32,
                                                class_mode='binary')

    test_set = test_datagen.flow_from_directory('dogs-vs-cats/',
                                           target_size = (64,64),
                                           batch_size = 32,
                                           class_mode ='binary')
    
#     training_set = set(itertools.islice(training_set, 100))
#     test_set = set(itertools.islice(test_set, 100))

    return training_set, test_set

#train_set, test_set = testAndTrain()
#train_set = np.asarray(train_labels).astype('float32').reshape((-1,1))
#y_test = np.asarray(test_labels).astype('float32').reshape((-1,1))
#input_shape = train_set[0][0].shape
(X_train, y_train), (X_test, y_test) = mnist.load_data()
img_size = X_train.shape[1]
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_size, img_size)
    X_test = X_test.reshape(X_test.shape[0], 1, img_size, img_size)
else:
    X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
    X_test = X_test.reshape(X_test.shape[0], img_size, img_size, 1)
X_train = X_train / 255

X_test = X_test / 255

X_train = X_train[:1000]
X_test = X_test[:550]
num_classes = 10
y_train = to_categorical(y_train[:1000], num_classes)
y_test = to_categorical(y_test[:550], num_classes)


In [370]:
X_test.shape

(550, 28, 28, 1)

In [371]:
# class Conv2D(Enum):
#     CONV2D_1 = 32
#     CONV2D_2 = 64
    
# class Pooling(Enum):
#     MAX_POOLING = 0.1
#     MIN_POOLING = 0.2
#     MEAN_POOLING = 0.5

# class Flatten(Enum):
#     FLATTEN = 1
    
# class DropOut(Enum):
#     DROP_OUT_1 = 0.25
#     DROP_OUT_2 = 0.20
#     DROP_OUT_3 = 0.50
    
# class Dense(Enum):
#     DENSE = 2
    

class Individual:
    def __init__(self, goal=100, numLayers=5):
        self.code = []
        self.dictionary = {}
        self.numLayers = numLayers
        
        self.dictionary[0] = Conv2D

        for i in range(1,2):
            pom = self.randomLayer()
            #print(type(pom))
            self.dictionary[i] = type(pom)
        pom = Flatten()
        self.dictionary[2] = type(pom)
        pom = Dense(units = 10, activation = 'softmax')
        self.dictionary[3] = type(pom)
      
       # print(Conv2D)
        #print(self.dictionary[0] == Conv2D)

#         self.dictionary[3] = Dense(units = 10, activation = 'softmax')
        
#         self.code.append(self.randomFirstConv2DLayer())
#         self.code.append(self.randomMiddleLayer())
#         self.code.append(Flatten()) 
#         self.code.append(Dense(units = 2, activation='softmax'))
        self.fitness = self.calcFitness()
        
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def __str__(self):
        modelString = ""
        for i in range(len(self.dictionary)):
            modelString+=" "
            modelString+=str(i)
        
        return str(self.dictionary)
    
    def calcFitness(self):
        
        model = tensorflow.keras.Sequential()
        model.add(self.randomFirstConv2DLayer())
        
        for i in range(1,len(self.dictionary)):
            if(self.dictionary[i]== Conv2D):
                print("uso1")
                model.add(self.randomConv2D())
            elif(self.dictionary[i]== MaxPooling2D):
                print("uso2")
                model.add(self.randomMaxPooling())
            elif(self.dictionary[i] == AveragePooling2D):
                print("uso3")
                model.add(self.randomAveragePooling())
            elif(self.dictionary[i]== Dropout):
                model.add(self.randomDropout())
            elif(self.dictionary[i]== Flatten):
                print("uso4")
                model.add(keras.layers.Flatten())
            else:
                print("uso5")
                model.add(keras.layers.Dense(units = 10 ,activation = 'softmax'))

                
        model.summary()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        model.fit(X_train, y_train, batch_size=128, epochs=3, validation_split=0.2)
        model.evaluate(X_test, y_test)
        
        y_pred = np.argmax(model.predict(X_test),axis = 1)
        y_true = np.argmax(y_test,axis = 1)
        print(confusion_matrix(y_true, y_pred))
        
        return accuracy_score(y_true, y_pred)

    def randomFiltersConv2D(self):
        return random.choice([32, 64])
    
    def randomKernelSizeConv2D(self):
        return random.choice([(3, 3)])
    
    def randomStrideConv2D(self):
        return random.choice([(1, 1)])
    
    def randomPaddingConv2D(self):
        return random.choice(["same"])
    
    def randomActivationConv2D(self):
        return random.choice(["relu"])
    
    def randomFirstConv2DLayer(self):
        layers = [keras.layers.Conv2D(32, kernel_size = self.randomKernelSizeConv2D(), strides = self.randomStrideConv2D(), padding = self.randomPaddingConv2D(), activation = self.randomActivationConv2D(), input_shape = (28,28,1)) for i in range(5)]
        return random.choice(layers)
    
    def randomConv2D(self):
        layers = [keras.layers.Conv2D(32 , kernel_size = self.randomKernelSizeConv2D(), strides = self.randomStrideConv2D(), padding = self.randomPaddingConv2D(), activation = self.randomActivationConv2D()) for i in range(5)]
        return random.choice(layers)
    
    def randomPoolSize(self):
        return random.choice([(2,2)])

    def randomMaxPooling(self):
        layers = [keras.layers.MaxPooling2D(pool_size = self.randomPoolSize()) for i in range(5)]
        return random.choice(layers)
    
    def randomAveragePooling(self):
        layers = [keras.layers.AveragePooling2D(pool_size = self.randomPoolSize()) for i in range(5)]
        return random.choice(layers)
    
    def randomDropout(self):
        layers = [keras.layers.Dropout(random.uniform(0,1)) for i in range(5)]
        return random.choice(layers)
    
    def randomLayer(self):
        return random.choice([self.randomConv2D(),self.randomMaxPooling(),self.randomAveragePooling(),self.randomDropout()])
    
    def randomMiddleLayer(self):
        layers = [self.randomLayer() for i in range(5)]
        return layers
    

        
  
   

In [372]:
def selection(population):
    TOURNAMENT_SIZE = 2
    participants = random.sample(population, TOURNAMENT_SIZE)
    winner = max(participants)
    return winner


In [373]:
def makeCNNOutOfLayers(layers,child):
    return true

In [374]:
def crossover(parent1, parent2, child1, child2):
    breakpoint1 = random.randrange(len(parent1.dictionary))
    breakpoint2 = random.randrange(len(parent2.dictionary))

    for i in range(breakpoint1):
        child1.dictionary[i] = parent1.dictionary[i]
        
    for i in range(breakpoint2, len(parent2.dictionary)):
        child1.dictionary[i] = parent2.dictionary[i]
        
    for i in range(breakpoint1, len(parent1.dictionary)):
        child2.dictionary[i] = parent1.dictionary[i]
        
    for i in range(breakpoint2):
        child2.dictionary[i] = parent2.dictionary[i]
        


In [375]:
def mutation(individual):
    MUTATION_PROB = 0.05
    for i in range(1,len(individual.code) - 2):
        if random.random() < MUTATION_PROB:
            individual.dictionary[i] = individual.randomLayer()

In [376]:
POP_SIZE = 5
NUM_GENERATIONS = 3
ELITISM_SIZE = POP_SIZE // 3
# if ELITISM_SIZE % 2 == 1:
#     ELITISM_SIZE -= 1

population = [Individual(goal=1381) for _ in range(POP_SIZE)]
newPopulation = [Individual(goal=1381) for _ in range(POP_SIZE)]


for i in range(NUM_GENERATIONS):
    population.sort(reverse=True)
    if population[0].fitness == 1:
        break
    newPopulation[:ELITISM_SIZE] = population[:ELITISM_SIZE]
    for j in range(ELITISM_SIZE, POP_SIZE, 2):
        parent1 = selection(population)
        parent2 = selection(population)
        
        crossover(parent1, parent2, newPopulation[j], newPopulation[j+1])
        
        mutation(newPopulation[j])
        mutation(newPopulation[j+1])
        
        newPopulation[j].fitness = newPopulation[j].calcFitness()
        newPopulation[j+1].fitness = newPopulation[j+1].calcFitness()
        
    population = newPopulation
    
best = max(population)
print(f'solution: {best}, fitness: {best.fitness}')

uso2
uso4
uso5
Model: "sequential_128"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1335 (Conv2D)         (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_480 (MaxPoolin (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_135 (Flatten)        (None, 6272)              0         
_________________________________________________________________
dense_158 (Dense)            (None, 10)                62730     
Total params: 63,050
Trainable params: 63,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
7/7 [==============================] - 0s 33ms/step - loss: 2.1356 - accuracy: 0.4837 - val_loss: 1.9112 - val_accuracy: 0.6800
Epoch 2/3
7/7 [==============================] - 0s 19ms/step - loss: 1.7194 - accuracy: 0.72

uso3
uso4
uso5
Model: "sequential_132"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1379 (Conv2D)         (None, 28, 28, 32)        320       
_________________________________________________________________
average_pooling2d_497 (Avera (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_143 (Flatten)        (None, 6272)              0         
_________________________________________________________________
dense_166 (Dense)            (None, 10)                62730     
Total params: 63,050
Trainable params: 63,050
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
7/7 [==============================] - 0s 33ms/step - loss: 2.2087 - accuracy: 0.3300 - val_loss: 2.0412 - val_accuracy: 0.5850
Epoch 2/3
7/7 [==============================] - 0s 15ms/step - loss: 1.8809 - accuracy: 0.68

18/18 [==============================] - 0s 5ms/step - loss: 0.5768 - accuracy: 0.8145
[[41  0  1  0  0  1  0  0  2  0]
 [ 0 68  0  0  0  2  0  0  2  0]
 [ 0  0 59  0  0  0  0  1  3  0]
 [ 0  0  5 26  0 19  0  2  2  0]
 [ 1  0  1  0 42  0  2  0  2 14]
 [ 1  0  0  0  0 46  0  1  4  2]
 [ 2  0  3  0  2  3 36  0  1  0]
 [ 0  1  0  0  2  0  0 43  1  4]
 [ 1  0  0  0  0  4  0  1 38  1]
 [ 0  1  1  0  1  4  0  0  1 49]]
uso3
uso4
uso5
Model: "sequential_137"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1437 (Conv2D)         (None, 28, 28, 32)        320       
_________________________________________________________________
average_pooling2d_526 (Avera (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_153 (Flatten)        (None, 6272)              0         
_________________________________________________________________
dense_176 (Dens

Epoch 1/3
7/7 [==============================] - 0s 39ms/step - loss: 2.1992 - accuracy: 0.3450 - val_loss: 1.9819 - val_accuracy: 0.7050
Epoch 2/3
7/7 [==============================] - 0s 19ms/step - loss: 1.8292 - accuracy: 0.7163 - val_loss: 1.6141 - val_accuracy: 0.7500
Epoch 3/3
18/18 [==============================] - 0s 3ms/step - loss: 1.3315 - accuracy: 0.7436
[[42  0  0  0  0  0  0  0  3  0]
 [ 0 68  0  1  0  0  0  0  1  2]
 [ 0  7 45  4  1  0  0  3  3  0]
 [ 0  2  2 45  0  1  1  1  1  1]
 [ 1  2  0  0 38  0  2  0  0 19]
 [ 3  1  0 14  3 24  2  1  4  2]
 [ 5  1  2  0  4  1 32  0  2  0]
 [ 0  4  0  0  1  0  0 42  1  3]
 [ 3  1  1  2  2  1  1  1 30  3]
 [ 0  2  0  2  7  0  0  1  2 43]]
uso4
uso5
Model: "sequential_142"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1467 (Conv2D)         (None, 28, 28, 32)        320       
________________________________________________________________

Epoch 1/3
7/7 [==============================] - 1s 89ms/step - loss: 1.9394 - accuracy: 0.4913 - val_loss: 1.1766 - val_accuracy: 0.7750
Epoch 2/3
7/7 [==============================] - 1s 73ms/step - loss: 0.7881 - accuracy: 0.8150 - val_loss: 0.6332 - val_accuracy: 0.8000
Epoch 3/3
18/18 [==============================] - 0s 5ms/step - loss: 0.5771 - accuracy: 0.8218
[[44  0  0  0  0  0  0  0  1  0]
 [ 0 68  0  0  0  3  0  0  1  0]
 [ 1  2 49  3  0  0  1  4  3  0]
 [ 0  0  0 44  0  6  1  2  1  0]
 [ 1  0  0  0 41  0  3  0  1 16]
 [ 3  0  0  1  1 44  0  2  2  1]
 [ 3  0  2  0  1  4 37  0  0  0]
 [ 0  1  0  0  0  0  0 48  0  2]
 [ 4  1  0  1  0  3  0  3 33  0]
 [ 0  1  0  4  1  1  0  5  1 44]]
uso4
uso5
Model: "sequential_147"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1504 (Conv2D)         (None, 28, 28, 32)        320       
________________________________________________________________